In [1]:
# import dependencies
import pandas as pd 
import gmaps 
import requests 

# Import API key 
from config import g_key

In [6]:
# configure gmaps to use API key 
gmaps.configure(api_key=g_key)

In [5]:
# Store the CSV into a DF 
city_data_df = pd.read_csv('weather_data/cities.csv')

In [14]:
# # get the max temp 
# max_temp = city_data_df['Max Temp']
# temps = []
# for temp in max_temp:
#     temps.append(max(temp,0))

In [21]:
# Heatmap of temp 
#get lat / long 
locations = city_data_df[['Lat','Lng']]

# Get the max temps 
max_temp = city_data_df['Max Temp']

# Assign the fig variable 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.2)

# Assign heatmap variable 
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer 
fig.add_layer(heat_layer)

# Call the Fig to plot the data 
fig 

Figure(layout=FigureLayout(height='420px'))

In [23]:
# HMap of % humidity 
locations = city_data_df[['Lat','Lng']]
humidity = city_data_df['Humidity']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# HMap of cloudiness
locations = city_data_df[['Lat','Lng']]
clouds = city_data_df['Cloudiness']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.2)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
# HMap of wind speed
locations = city_data_df[['Lat','Lng']]
wind = city_data_df['Wind Speed']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.2)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
# Ask the cust to add min and max temp values 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [31]:
# Filter the data to find cities matching criteria 
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                      (city_data_df['Max Temp'] >= min_temp)]

In [32]:
# Createa DF to store hotel names along with other data 
hotel_df = preferred_cities_df[['City','Country','Max Temp','Lat','Lng']].copy()
hotel_df['Hotel Name'] = "" 
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Tingo Maria,PE,79.11,-9.2895,-76.0088,
4,Funadhoo,MV,81.23,6.1500,73.2667,
5,Bull Savanna,JM,80.46,17.8868,-77.5902,
7,Uribia,CO,85.35,11.7139,-72.2660,
12,Tutoia,BR,85.98,-2.7619,-42.2744,


In [ ]:
# Set parameters to search for a hotel 
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

In [38]:
# iterate through the DF 
for index, row in hotel_df.iterrows():
    # Get the lat + long 
    lat = row['Lat']
    lng = row['Lng']
    
    # Add the lat / lng to a location key 
    params['location'] = f'{lat},{lng}'
    
    # Use the search term lodging and our lat + lng 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make requests and get JSON data 
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [41]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [43]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [44]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))